## <center>**App Reviews Multi-labels Classification**</center>

### **1. Construct the Task Prompt**

In [1]:
from utils.fileUtils import *

In [2]:
data_lines = readCsvToList('./test_data_android.csv')

In [3]:
assert len(data_lines) == 500

In [4]:
nfr_labels = ['Per','Usa','Sup','Dep','Mis']

In [5]:
data_lines[0]

["I had songs on my sd card that are still listed but won't play because for some reason all of a sudden those songs don't register as being on my sd card. If it happens again I will uninstall it",
 '0',
 '1',
 '0',
 '0',
 '0']

In [6]:
data_lines[-1]

['App spontaneously moves to advertising not selected by users. Even if we must see advertising on app pages, we should NEVER be transferred to ad sites without intentional selection. Further, when multiple watches are in place, why does wundermap only show pop-up least severe?',
 '0',
 '1',
 '0',
 '0',
 '0']

In [7]:
import openai

In [8]:
openai.api_key = 'YOUR_OWN_API_KEY'

In [9]:
# ['Per','Usa','Sup','Dep','Mis']
test_req = data_lines[15]
print(test_req[0],test_req[1:])

This used to be a great app, designed for one purpose, music. Now woth all of the additional add ons even songs that I have downloaded need to buffer and the musoc keeps on skipping. Amazon, change this app back to its original purpose and stopp adding stupid gimmicks. ['1', '1', '0', '0', '0']


In [10]:
prompt = '''Does the following App review involve non-functional requirements about usability:\n\n'''

In [11]:
response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt + test_req[0]}],
        temperature = 0,
        top_p = 1
    )

In [12]:
content = response['choices'][0]['message']['content']

In [13]:
content.strip()

"Yes, the review involves non-functional requirements about usability, specifically about the app's performance and ease of use. The user is complaining about the app's ability to play music smoothly and without interruption, which is a usability issue."

In [14]:
print(prompt + test_req[0])

Does the following App review involve non-functional requirements about usability:

This used to be a great app, designed for one purpose, music. Now woth all of the additional add ons even songs that I have downloaded need to buffer and the musoc keeps on skipping. Amazon, change this app back to its original purpose and stopp adding stupid gimmicks.


### **2. Multi-labels Classification on App Reviews with ChatGPT**

In [15]:
def query_ChatGPT(req_str):
    prompt = '''Does the following App review involve non-functional requirements about usability:\n\n'''
    response = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        messages = [{"role": "user", "content": prompt + req_str}],
        temperature = 0,
        top_p = 1
    )
    content = response['choices'][0]['message']['content']
    label = content.strip()
    return label

In [16]:
import time
import logging
from datetime import datetime

# current time
now = datetime.now()
# current time string
time_str = now.strftime("%Y-%m-%d %H:%M:%S")

logging.basicConfig(level=logging.INFO,
                    filename='./logs_android/{}-{}.log'.format('usability',time_str),
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')

In [17]:
data_lines = readCsvToList('./test_data_android.csv')

In [18]:
# save logging information and results to local files (writing file with `append` mode)
for idx,line in enumerate(data_lines):
    req = line[0].strip()
    
    while True:
        try:
            pre = query_ChatGPT(req)
            logging.info(idx+1)
            with open('./results_android/{}-{}.txt'.format('usability',time_str),'a+') as f:
                f.write(str(idx+1)+' -*- '+pre.strip().replace('\n',' ')+'\n')
            break
        except:
            time.sleep(3)
            continue